Instructions
-------------

This notebook carries out the experiment on simulated data in Sect. 6.1 of our paper "Pragmatic Causal Feature Learning". The experiment proceeds in two stages, first deriving a coarsening using Chalupka et al.'s CFL method, before using the same data to derive a different coarsening using our PCFL method. 

In [ ]:
""" This cell sets up the Python environment. Make sure that you have the following packages installed:
numpy
sklearn
"""

import sys

import numpy as np
np.random.seed(1423)
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
#Set the number of data points and the range of C and E. Note that some steps can take a while when m>10000, and that our code is prohibitively slow at m=1000000. 
m=10000
R_C = np.array([2,4,6,8])
R_E = np.array([2,4,6,8])

#Generate data using the joint probability distribution over CxE.
pairs = np.array(np.transpose([np.tile(R_C, len(R_E)), np.repeat(R_E, len(R_C))]))
probs = np.array([1/20,1/40,97/400,1/20,1/40,3/80,1/343,1/40,1/40,3/80,1/343,1/40,3/20,3/20,1/600,3/20])
probs[15] = probs[15]+(1-sum(probs))
data_indeces = np.array([np.random.choice(np.arange(0,16),p=probs) for i in range(0,m)])
data = np.array([pairs[data_indeces[i]] for i in range(0,m)])

In [626]:
#Define X and Y vectors based on the data set.

X = np.array([data[i][0] for i in range(0,m)]).reshape(-1,1)
Y = np.array([data[i][1] for i in range(0,m)]).reshape(-1,1)

#Define a utility function over XxY, and record utilities in a vector Z.
def u(x,y):
    if y!=8:
        return 100-x*y
    else:
        return 100-(x*(y-2))
    
Z = np.array([u(data[i][0],data[i][1]) for i in range(0,m)]).reshape(-1,1)

CFL
-------------

The next five cells implement Chalupka et al.'s CFL algorithm on the data generated above (Alg. 1 in our paper) to learn a three-valued coarsening of the cause and effect variables.

In [627]:
#Regress X on Y using cubic regression. 
model = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression(fit_intercept=False))])
model = model.fit(X, Y)
X_estimator = np.array([model.named_steps['linear'].coef_[0][0] + model.named_steps['linear'].coef_[0][1]*X[i] + model.named_steps['linear'].coef_[0][2]*X[i]**2 + model.named_steps['linear'].coef_[0][3]*X[i]**3 for i in range(0,m)])

In [628]:
#Cluster Xs.
N_CLASSES = 3
X_lbls = KMeans(n_clusters=N_CLASSES, n_init=10, n_jobs=-1).fit_predict(X_estimator.reshape(-1,1))

In [629]:
#Cluster Ys.
Y_ftrs = np.zeros((Y.shape[0], np.unique(X_lbls).size))
# Loop, not vectorized, to save memory. Can take a while.
for Y_id, y in enumerate(Y):
    if Y_id % 100==0:
        sys.stdout.write('\rComputing P(y | X_lbls) features, iter {}/{}...'.format(Y_id, Y.shape[0]))
        sys.stdout.flush() 
    for X_lbl_id, X_lbl in enumerate(np.unique(X_lbls)):
        # Find ids of xs in this X_lbls class.
        X_lbl_ids = np.where(X_lbls==X_lbl)[0]
        # Compute distances of y to all y's in this X_lbls class and sort them.
        sorted_dists = np.sort([np.sum((y-Y[X_lbl_ids[i]])**2) for i in range(0,len(X_lbl_ids))])
        # Find the mean distance to the 4 closest non-zero points.
        Non_Zero_Distance = np.where(sorted_dists!=0)[0]
        Y_ftrs[Y_id][X_lbl_id] = Non_Zero_Distance[1:5].mean()
print('Done. Clustering P(y | X_lbls).')
Y_lbls = KMeans(n_clusters=N_CLASSES, n_init=10, n_jobs=-1).fit_predict(Y_ftrs)

Computing P(y | X_lbls) features, iter 9900/10000...Done. Clustering P(y | X_lbls).


In [630]:
#Create labels showing how variables have been coarsened.

full_matrix=np.hstack((data,X_lbls.reshape(-1,1),Y_lbls.reshape(-1,1)))

crs_C0 = np.delete(full_matrix,np.where(full_matrix[:,2]!=0),axis=0)
crs_C0 = crs_C0[:,0].reshape(-1,1)
crs_C0 = np.array([crs_C0[i] for i in range(0,len(crs_C0))])
crs_C0 = list(np.unique(crs_C0))
crs_C1 = np.delete(full_matrix,np.where(full_matrix[:,2]!=1),axis=0)
crs_C1 = crs_C1[:,0].reshape(-1,1)
crs_C1 = np.array([crs_C1[i] for i in range(0,len(crs_C1))])
crs_C1 = list(np.unique(crs_C1))
crs_C2 = np.delete(full_matrix,np.where(full_matrix[:,2]!=2),axis=0)
crs_C2 = crs_C2[:,0].reshape(-1,1)
crs_C2 = np.array([crs_C2[i] for i in range(0,len(crs_C2))])
crs_C2 = list(np.unique(crs_C2))

crs_E0 = np.delete(full_matrix,np.where(full_matrix[:,3]!=0),axis=0)
crs_E0 = crs_E0[:,1].reshape(-1,1)
crs_E0 = np.array([crs_E0[i] for i in range(0,len(crs_E0))])
crs_E0 = list(np.unique(crs_E0))
crs_E1 = np.delete(full_matrix,np.where(full_matrix[:,3]!=1),axis=0)
crs_E1 = crs_E1[:,1].reshape(-1,1)
crs_E1 = np.array([crs_E1[i] for i in range(0,len(crs_E1))])
crs_E1 = list(np.unique(crs_E1))
crs_E2 = np.delete(full_matrix,np.where(full_matrix[:,3]!=2),axis=0)
crs_E2 = crs_E2[:,1].reshape(-1,1)
crs_E2 = np.array([crs_E2[i] for i in range(0,len(crs_E2))])
crs_E2 = list(np.unique(crs_E2))

In [634]:
#Find the conditional probability table and print, with labels. Note the guide after the table showing which fine-grained variable values have been coarsened together. 

P_CE = np.array([np.bincount(Y_lbls.astype(int)[X_lbls==X_lbl],minlength=Y_lbls.max()+1).astype(float) for X_lbl in np.sort(np.unique(X_lbls))])
P_CE = P_CE/P_CE.sum()
P_E_given_C = np.around(P_CE/P_CE.sum(axis=1, keepdims=True),2)
column = np.array(['crs_C0','crs_C1','crs_C2']).reshape(-1,1)
header = np.array(['    ','crs_E0','crs_E1','crs_E2'])
P_E_given_C = np.hstack((column,P_E_given_C))
P_E_given_C = np.vstack((header,P_E_given_C))
print(P_E_given_C)
print('crs_C0=', crs_C0)
print('crs_C1=', crs_C1)
print('crs_C2=', crs_C2)
print('crs_E0=', crs_E0)
print('crs_E1=', crs_E1)
print('crs_E2=', crs_E2)

[['    ' 'crs_E0' 'crs_E1' 'crs_E2']
 ['crs_C0' '0.6' '0.11' '0.3']
 ['crs_C1' '0.01' '0.97' '0.02']
 ['crs_C2' '0.6' '0.21' '0.19']]
crs_C0= [4]
crs_C1= [6]
crs_C2= [2, 8]
crs_E0= [8]
crs_E1= [2]
crs_E2= [4, 6]


PCFL
-------------

The next five cells implement our PCFL algorithm on the data generated above (Alg. 2 in our paper) to learn a three-valued pragmatic coarsening of the cause and effect variables.

In [635]:
#Regress X on the vector Z of utility values u(x,y)

model = Pipeline([('poly', PolynomialFeatures(degree=3)),('linear', LinearRegression(fit_intercept=False))])
model = model.fit(X, Z)
X_estimator_prag = np.array([model.named_steps['linear'].coef_[0][0] + model.named_steps['linear'].coef_[0][1]*X[i] + model.named_steps['linear'].coef_[0][2]*X[i]**2 + model.named_steps['linear'].coef_[0][3]*X[i]**3 for i in range(0,m)])

In [636]:
# Cluster Xs.
N_CLASSES = 3
X_lbls_prag = KMeans(n_clusters=N_CLASSES, n_init=10, n_jobs=-1).fit_predict(X_estimator_prag.reshape(-1,1))

In [637]:
#Cluster Ys. Takes less time than this step does when implementing CFL. 
Y_ftrs = np.zeros((Y.shape[0],m))
for i in range(0,m):
        helper_matrix = np.hstack((data,Z))
        helper_matrix = np.delete(helper_matrix,np.where(helper_matrix[:,1]!=Y[i]),axis=0)
        Y_ftrs[i] = helper_matrix[:,2].mean()
print('Done. Clustering P(y | X_lbls).')
Y_lbls_prag = KMeans(n_clusters=N_CLASSES, n_init=10, n_jobs=-1).fit_predict(Y_ftrs)

Done. Clustering P(y | X_lbls).


In [638]:
#Create labels showing how variables have been coarsened.

full_matrix=np.hstack((data,X_lbls_prag.reshape(-1,1),Y_lbls_prag.reshape(-1,1)))

crs_C0 = np.delete(full_matrix,np.where(full_matrix[:,2]!=0),axis=0)
crs_C0 = crs_C0[:,0].reshape(-1,1)
crs_C0 = np.array([crs_C0[i] for i in range(0,len(crs_C0))])
crs_C0 = list(np.unique(crs_C0))
crs_C1 = np.delete(full_matrix,np.where(full_matrix[:,2]!=1),axis=0)
crs_C1 = crs_C1[:,0].reshape(-1,1)
crs_C1 = np.array([crs_C1[i] for i in range(0,len(crs_C1))])
crs_C1 = list(np.unique(crs_C1))
crs_C2 = np.delete(full_matrix,np.where(full_matrix[:,2]!=2),axis=0)
crs_C2 = crs_C2[:,0].reshape(-1,1)
crs_C2 = np.array([crs_C2[i] for i in range(0,len(crs_C2))])
crs_C2 = list(np.unique(crs_C2))

crs_E0 = np.delete(full_matrix,np.where(full_matrix[:,3]!=0),axis=0)
crs_E0 = crs_E0[:,1].reshape(-1,1)
crs_E0 = np.array([crs_E0[i] for i in range(0,len(crs_E0))])
crs_E0 = list(np.unique(crs_E0))
crs_E1 = np.delete(full_matrix,np.where(full_matrix[:,3]!=1),axis=0)
crs_E1 = crs_E1[:,1].reshape(-1,1)
crs_E1 = np.array([crs_E1[i] for i in range(0,len(crs_E1))])
crs_E1 = list(np.unique(crs_E1))
crs_E2 = np.delete(full_matrix,np.where(full_matrix[:,3]!=2),axis=0)
crs_E2 = crs_E2[:,1].reshape(-1,1)
crs_E2 = np.array([crs_E2[i] for i in range(0,len(crs_E2))])
crs_E2 = list(np.unique(crs_E2))

In [640]:
#Find the conditional probability table and print, with labels. Note the guide after the table showing which fine-grained variable values have been coarsened together. 
P_CE = np.array([np.bincount(Y_lbls_prag.astype(int)[X_lbls_prag==X_lbl],minlength=Y_lbls_prag.max()+1).astype(float) for X_lbl in np.sort(np.unique(X_lbls_prag))])
P_CE = P_CE/P_CE.sum()
P_E_given_C = np.around(P_CE/P_CE.sum(axis=1, keepdims=True),2)
column = np.array(['crs_C0','crs_C1','crs_C2']).reshape(-1,1)
header = np.array(['    ','crs_E0','crs_E1','crs_E2'])
P_E_given_C = np.hstack((column,P_E_given_C))
P_E_given_C = np.vstack((header,P_E_given_C))
print(P_E_given_C)
print('crs_C0=', crs_C0)
print('crs_C1=', crs_C1)
print('crs_C2=', crs_C2)
print('crs_E0=', crs_E0)
print('crs_E1=', crs_E1)
print('crs_E2=', crs_E2)

[['    ' 'crs_E0' 'crs_E1' 'crs_E2']
 ['crs_C0' '0.59' '0.36' '0.05']
 ['crs_C1' '0.21' '0.69' '0.1']
 ['crs_C2' '0.11' '0.74' '0.15']]
crs_C0= [2, 6]
crs_C1= [8]
crs_C2= [4]
crs_E0= [2]
crs_E1= [6, 8]
crs_E2= [4]
